In [7]:
import pandas as pd
import numpy as np

from time import time
from scipy.stats import randint as sp_randint

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
#from dask_searchcv import GridSearchCV
#from dask_searchcv import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier

In [8]:


df = pd.read_csv('./data/feature_engineered.csv')
df = df.dropna()

df.sample(5)


,loan_type_str_VA-guarenteed,loan_type_str_Conventional,loan_type_str_FHA-Insured,loan_type_str_FSA/RHS,property_type_str_1 to 4 Family - non manufactured,property_type_str_Manufactured Housing,property_type_str_Multifamily,loan_purpose_str_Home purchase,loan_purpose_str_Refinancing,loan_purpose_str_Home improvement,...,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner-occupied_units,number_of_1_to_4_family_units,lender,co_applicant,accepted,msa_md_str,state_code_str,county_code_str
73253,0,1,0,0,1,0,0,0,1,0,...,-0.867542,0.4604,-1.063397,-0.191468,2839,0,0,180.0,25.0,58.0
178506,0,1,0,0,1,0,0,0,1,0,...,0.681818,1.0000,0.252392,-0.213294,6240,0,0,305.0,47.0,68.0
405141,0,1,0,0,1,0,0,0,1,0,...,0.565429,1.0000,-0.113636,-0.426587,5316,1,0,305.0,47.0,68.0
341033,0,1,0,0,1,0,0,1,0,0,...,-0.209795,1.0000,-0.632775,-0.583333,3723,0,0,237.0,42.0,288.0
431132,0,1,0,0,1,0,0,1,0,0,...,-0.017286,1.0000,-0.167464,-0.481151,5498,0,1,383.0,25.0,49.0


In [9]:
# reduced features
'''
[(0.0966, 'lender'), (0.0963, 'applicant_income'), (0.0949, 'loan_amount'), (0.0848, 'minority_population_pct'), (0.0825, 'ffiecmedian_family_income'), (0.0777, 'population'), (0.0766, 'number_of_owner-occupied_units'), (0.0762, 'number_of_1_to_4_family_units'), (0.0572, 'county_code'),
(0.0544, 'loan_purpose'), (0.0489, 'msa_md'), (0.0445, 'state_code'), (0.0383, 'tract_to_msa_md_income_pct'), (0.0148, 'applicant_race'), (0.0133, 'loan_type'), (0.0126, 'property_type'), (0.0106, 'co_applicant'), (0.0105, 'applicant_ethnicity'), (0.0094, 'preapproval')]
'''
# reduced_feature_df = df[['accepted','lender','applicant_income', 'loan_amount', 'minority_population_pct','ffiecmedian_family_income', 'population', 'number_of_owner-occupied_units','number_of_1_to_4_family_units', 'loan_purpose','co_applicant', 'loan_type'  ]]

reduced_feature_df =  df #df[['accepted','applicant_income', 'loan_amount', 'state_code','ffiecmedian_family_income',  'loan_type'  ]]

reduced_feature_df.sample(5)

,loan_type_str_VA-guarenteed,loan_type_str_Conventional,loan_type_str_FHA-Insured,loan_type_str_FSA/RHS,property_type_str_1 to 4 Family - non manufactured,property_type_str_Manufactured Housing,property_type_str_Multifamily,loan_purpose_str_Home purchase,loan_purpose_str_Refinancing,loan_purpose_str_Home improvement,...,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner-occupied_units,number_of_1_to_4_family_units,lender,co_applicant,accepted,msa_md_str,state_code_str,county_code_str
267971,0,1,0,0,1,0,0,1,0,0,...,-0.912612,1.00000,0.468900,0.177579,84,1,1,351.0,37.0,251.0
140714,0,1,0,0,1,0,0,0,0,1,...,0.134827,1.00000,0.332536,0.241071,2166,1,0,333.0,6.0,281.0
439835,0,1,0,0,1,0,0,0,1,0,...,0.607490,0.53042,-1.348086,-1.302579,5735,0,1,131.0,1.0,163.0
254445,0,1,0,0,1,0,0,0,0,1,...,-0.778809,0.67513,-0.900718,-0.663690,882,0,0,386.0,37.0,104.0
254986,0,1,0,0,1,0,0,1,0,0,...,2.679257,0.46968,-0.967703,-1.095238,684,0,1,322.0,37.0,100.0


In [10]:
# split up the data into labels and features
labels = reduced_feature_df['accepted']
features = reduced_feature_df.drop(columns=['accepted'])

# split test / train data
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import datetime

# first pass - standard 50/50 split
train, test, train_labels, test_labels = train_test_split(features,
                                                          labels,
                                                          test_size=0.30,
                                                          random_state=131221)



In [11]:
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_validate
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import confusion_matrix

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# from sklearn.svm import LinearSVC

#cv_results = cross_validate(lasso, X, y, cv=3, return_train_score=False)


def train_compare_model( model , train, train_labels, test, test_labels ):
    model.fit(train, train_labels )
    preds = model.predict(test)
    col_names = train.columns.values

    print('accuracy of model is:',accuracy_score(test_labels, preds))
    print("Features sorted by their score:")
    zipped_objs = zip(map(lambda x: round(x, 4), bdt.feature_importances_), col_names)
    print( sorted(zipped_objs, reverse=True)[0:10] )
    
    #cv_results = cross_validate(model , X, y, cv=3, return_train_score=False)



In [ ]:
c_train, c_test, c_train_labels, c_test_labels = train_test_split(features,
                                                          labels,
                                                          test_size=0.00,
                                                          random_state=9918272)

bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2),
                         algorithm="SAMME",
                         n_estimators=200)

bdt.fit(c_train, c_train_labels )
preds = bdt.predict(c_test)
col_names = train.columns.values

print('accuracy of model is:',accuracy_score(c_test_labels, preds))
print("Features sorted by their score:")
zipped_objs = zip(map(lambda x: round(x, 4), bdt.feature_importances_), col_names)
print( sorted(zipped_objs, reverse=True)[0:10] )
    
cv_results = cross_validate(bdt , c_train, c_train_labels, cv=5, return_train_score=True)

print('log')
print(cv_results)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2),
                         algorithm="SAMME",
                         n_estimators=200)

train_compare_model( bdt, train, train_labels, test , test_labels )

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
import matplotlib.pyplot as plt
import numpy as np

# create a new classifiser
rfc = RandomForestClassifier()
train_compare_model( bdt, train, train_labels, test,  test_labels )

'''
# Train the model
model2 = rfc.fit(train, train_labels)
preds2 = rfc.predict(test)
col_names = reduced_feature_df.columns.values
'''


print('a')

In [ ]:

#diabetes = datasets.load_diabetes()
#X = diabetes.data[:150]
#>>> y = diabetes.target[:150]
#>>> lasso = linear_model.Lasso()

In [ ]:
# Isolate feature importances 
importance = rfc.feature_importances_

# Sort the feature importances 
sorted_importances = np.argsort(importance)

# Insert padding
padding = np.arange(len(col_names)-1) + 0.5

# Plot the data
plt.barh(padding, importance[sorted_importances], align='center')

# Customize the plot
plt.yticks(padding, col_names[sorted_importances])
plt.xlabel("Relative Importance")
plt.title("Variable Importance")

# Show the plot
plt.show()

In [ ]:
# export the model for use in other notebooks
import pickle

filename = 'adaboost_rtc_v2.sav'
pickle.dump(bdt, open(filename, 'wb'))

In [ ]:
'''
# build a classifier
clf = RandomForestClassifier(n_estimators=20)


# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5)

start = time()
#model2 = rfc.fit(train, train_labels)
random_search.fit(train, train_labels)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

# use a full grid over all parameters
param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5)
start = time()
grid_search.fit(train, train_labels)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)
'''